In [1]:
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import sys
import numpy.linalg as linalg
import numpy as np
import seaborn as sns
import pandas as pd
import scipy
from networkx.algorithms import bipartite
import sympy as sp

In [2]:
def show_matrix(data):
    min_val, max_val = 0, 15
    cm = sns.light_palette("blue", as_cmap=True)
    x=pd.DataFrame(data)
    x=x.style.background_gradient(cmap=cm)
    display(x)

In [3]:
def calculate_second_egein(norm_Lapl):
    val, vec = np.linalg.eigh(norm_Lapl)
    val = val.real
    vec = vec.real
    idx = np.argsort(val)
    val = val[idx]
    vec = vec[:,idx]
    return val,vec

In [4]:
def create_graph():
    G = nx.Graph()
    G.add_nodes_from([1, 2, 3, 4, 5, 6, 7, 8])
    G.add_edges_from([(1, 2), (1, 3), (1, 4), (2, 3), (2, 4), (3, 4), (4, 5), (5, 6), (5, 7), (5, 8), (6, 7), (7, 8), (6, 8)])
    return G

In [5]:
def show_graph(G, colors):
    plt.figure(figsize=(15, 15))
    plt.subplot(121)
    nx.draw(G, node_color=colors, with_labels=True, font_weight='bold')
    plt.show()

In [6]:
def show_graph(G):
    plt.figure(figsize=(15, 15))
    plt.subplot(121)
    nx.draw(G, with_labels=True, font_weight='bold')
    plt.show()

In [7]:
def get_not_normal_eigen(L):
    mat = sp.Matrix(L)
    tupels = mat.eigenvects()
    
    values = []
    vectors = []

    for eigen_value, _, eigen_vector in tupels:
        for v in eigen_vector:
            values.append(float(eigen_value))
            _v = []
            for __v in np.array(v).astype(np.float64):
                _v.append(__v[0])
            vectors.append(_v)

    return values, vectors

In [8]:
G = create_graph()

In [9]:
show_graph(G)

NameError: name 'colors' is not defined

In [9]:
A = nx.convert_matrix.to_numpy_array(G, dtype=int)
show_matrix(A)

,0,1,2,3,4,5,6,7
0,0,1,1,1,0,0,0,0
1,1,0,1,1,0,0,0,0
2,1,1,0,1,0,0,0,0
3,1,1,1,0,1,0,0,0
4,0,0,0,1,0,1,1,1
5,0,0,0,0,1,0,1,1
6,0,0,0,0,1,1,0,1
7,0,0,0,0,1,1,1,0


In [10]:
DergeeList = list(G.degree)
D = np.zeros((G.number_of_nodes(), G.number_of_nodes()),dtype=int)
for node in range(len(DergeeList)):
    D[node][node] = DergeeList[node][1]

show_matrix(D)

,0,1,2,3,4,5,6,7
0,3,0,0,0,0,0,0,0
1,0,3,0,0,0,0,0,0
2,0,0,3,0,0,0,0,0
3,0,0,0,4,0,0,0,0
4,0,0,0,0,4,0,0,0
5,0,0,0,0,0,3,0,0
6,0,0,0,0,0,0,3,0
7,0,0,0,0,0,0,0,3


In [11]:
Lap = D - A
show_matrix(Lap)

,0,1,2,3,4,5,6,7
0,3,-1,-1,-1,0,0,0,0
1,-1,3,-1,-1,0,0,0,0
2,-1,-1,3,-1,0,0,0,0
3,-1,-1,-1,4,-1,0,0,0
4,0,0,0,-1,4,-1,-1,-1
5,0,0,0,0,-1,3,-1,-1
6,0,0,0,0,-1,-1,3,-1
7,0,0,0,0,-1,-1,-1,3


In [12]:
eiegen_vals, eigen_vecs = get_not_normal_eigen(Lap)
print (eiegen_vals)
print (eigen_vecs)

[0.0, 4.0, 4.0, 4.0, 4.0, 4.0, 0.3542486889354094, 5.645751311064591]
[[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0], [-1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [-1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0], [1.0, 0.0, 0.0, -1.0, -1.0, 1.0, 0.0, 0.0], [1.0, 0.0, 0.0, -1.0, -1.0, 0.0, 1.0, 0.0], [1.0, 0.0, 0.0, -1.0, -1.0, 0.0, 0.0, 1.0], [-1.0, -1.0, -1.0, -0.6457513110645906, 0.6457513110645906, 1.0, 1.0, 1.0], [-1.0, -1.0, -1.0, 4.645751311064591, -4.645751311064591, 1.0, 1.0, 1.0]]


In [13]:
# evals,evecs = calculate_second_egein(Lap)
second_vector = eigen_vecs[0]
label_vector = np.zeros(len(second_vector))
print ("label_vector: ")
print(label_vector)
median = np.median(second_vector)
print ("median: ")
print (median)
label_vector[second_vector>median]=1
label_vector[second_vector<=median]=0
print ("label_vector2: ")
print(label_vector)
color_map = []
for i in label_vector:
    if i == 0:
        color_map.append('red')
    else:
        color_map.append('green')
print ("label vector")
print (label_vector)

label_vector: 
[0. 0. 0. 0. 0. 0. 0. 0.]
median: 
1.0
label_vector2: 
[0. 0. 0. 0. 0. 0. 0. 0.]
label vector
[0. 0. 0. 0. 0. 0. 0. 0.]


In [14]:
plt.figure(figsize=(8, 8))
nx.draw_networkx_edges(G, pos, nodelist=[ncenter], alpha=0.4)
nx.draw_networkx_nodes(
    G,
    pos,
    nodelist=list(p.keys()),
    node_size=80,
    node_color=color_map,
    cmap=plt.cm.Reds_r,
)
nx.draw_networkx_labels(G,pos,labels,font_size=7)
plt.xlim(-0.05, 1.05)
plt.ylim(-0.05, 1.05)
plt.axis("off")
plt.show()

NameError: name 'pos' is not defined

In [16]:
second_vector = eigen_vecs[0]
label_vector = np.zeros(len(second_vector))
print ("label_vector: ")
print(label_vector)
median = np.median(second_vector)
print ("median: ")
print (median)
label_vector[second_vector>median]=1
label_vector[second_vector<=median]=0
print ("label_vector2: ")
print(label_vector)
color_map = []
for i in label_vector:
    print(i)
    if i == 0:
        color_map.append('red')
    else:
        color_map.append('green')
print ("label vector")
print (label_vector)

label_vector: 
[0. 0. 0. 0. 0. 0. 0. 0.]
median: 
1.0
label_vector2: 
[0. 0. 0. 0. 0. 0. 0. 0.]
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
label vector
[0. 0. 0. 0. 0. 0. 0. 0.]


In [17]:
evals,evecs = get_not_normal_eigen(Lap)
# print(evals)
idx = np.argsort(evals)
print(idx)
val = evals[idx]
vec = evecs[idx[1]]
print (val)
print (vec)

[0 6 1 2 3 4 5 7]


TypeError: only integer scalar arrays can be converted to a scalar index

In [20]:
L_norm = nx.normalized_laplacian_matrix(G)
#evals,evecs = calculate_eigen(Lap)
evals,evecs = get_not_normal_eigen(Lap)
print(evals)
print(evecs)
# idx = np.argsort(evals)
# val = evals[idx]
# vec = evecs[idx]
# print (val)
# print (vec)
#print ("Sorted vectors")
# show_vec_array(evecs,1)
print ("Non sorted vectors")
# show_vec_array(evecs,0)
second_vector = evecs[6]
print ("evals")
print (evals)
print (np.transpose(second_vector))
label_vector = np.zeros(len(second_vector))
median = np.median(second_vector)
print ("median: ")
print (median)
label_vector[second_vector>median]=1
label_vector[second_vector<median]=0
color_map = []
for i in label_vector:
    if i == 0:
        color_map.append('red')
    else:
        color_map.append('green')
print ("label vector")
print (label_vector)

[0.0, 4.0, 4.0, 4.0, 4.0, 4.0, 0.3542486889354094, 5.645751311064591]
[[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0], [-1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [-1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0], [1.0, 0.0, 0.0, -1.0, -1.0, 1.0, 0.0, 0.0], [1.0, 0.0, 0.0, -1.0, -1.0, 0.0, 1.0, 0.0], [1.0, 0.0, 0.0, -1.0, -1.0, 0.0, 0.0, 1.0], [-1.0, -1.0, -1.0, -0.6457513110645906, 0.6457513110645906, 1.0, 1.0, 1.0], [-1.0, -1.0, -1.0, 4.645751311064591, -4.645751311064591, 1.0, 1.0, 1.0]]
Non sorted vectors
evals
[0.0, 4.0, 4.0, 4.0, 4.0, 4.0, 0.3542486889354094, 5.645751311064591]
[-1.         -1.         -1.         -0.64575131  0.64575131  1.
  1.          1.        ]
median: 
0.0
label vector
[0. 0. 0. 0. 1. 1. 1. 1.]


In [21]:
plt.figure(figsize=(8, 8))
draw_graph(G,p,pos,ncenter,labels,color_map,0)
plt.axis("off")
plt.show()

NameError: name 'draw_graph' is not defined